In [130]:
import json
import requests
from pprint import pprint
import pandas as pd

In [131]:
base_url = 'http://lookup-service-prod.mlb.com'

In [132]:
all_teams_endpoint = "/json/named.team_all_season.bam?sport_code='mlb'&all_star_sw='N'&sort_order=name_asc&season='2019'"

In [133]:
all_teams_response = requests.get(base_url + all_teams_endpoint)
print(all_teams_response)
print(all_teams_response.url)

<Response [200]>
http://lookup-service-prod.mlb.com/json/named.team_all_season.bam?sport_code='mlb'&all_star_sw='N'&sort_order=name_asc&season='2019'


In [134]:
all_teams_data = all_teams_response.json()
pprint(all_teams_data)

{'team_all_season': {'copyRight': ' Copyright 2019 MLB Advanced Media, L.P.  '
                                  'Use of any content on this page '
                                  'acknowledges agreement to the terms posted '
                                  'here '
                                  'http://gdx.mlb.com/components/copyright.txt  ',
                     'queryResults': {'created': '2019-05-15T07:31:55',
                                      'row': [{'active_sw': 'Y',
                                               'address': '401 East Jefferson '
                                                          'Street&#xa;Phoenix, '
                                                          'AZ&#x9;85004',
                                               'address_city': 'Phoenix',
                                               'address_country': '',
                                               'address_intl': 'N',
                                               'address_line1':

                                               'store_url': 'losangeles.dodgers.mlb.com',
                                               'team_code': 'lan',
                                               'team_id': '119',
                                               'time_zone': 'PT',
                                               'time_zone_alt': 'America/Los_Angeles',
                                               'time_zone_generic': 'PT',
                                               'time_zone_num': '-7',
                                               'time_zone_text': 'PDT',
                                               'venue_id': '22',
                                               'venue_name': 'Dodger Stadium',
                                               'venue_short': 'Dodger Stadium',
                                               'website_url': 'dodgers.com'},
                                              {'active_sw': 'Y',
                                          

                                               'address_line2': '',
                                               'address_line3': '',
                                               'address_province': '',
                                               'address_state': 'FL',
                                               'address_zip': '33705',
                                               'all_star_sw': 'N',
                                               'base_url': 'rays.mlb.com',
                                               'bis_team_code': 'TB',
                                               'city': 'Tampa Bay',
                                               'division': 'E',
                                               'division_abbrev': 'ALE',
                                               'division_full': 'American '
                                                                'League East',
                                               'division_id': '201',
           

In [135]:
all_teams_data_for_df = all_teams_data['team_all_season']['queryResults']['row']

In [136]:
teams_df = pd.DataFrame(all_teams_data_for_df)
teams_df.head()

,active_sw,address,address_city,address_country,address_intl,address_line1,address_line2,address_line3,address_province,address_state,...,team_id,time_zone,time_zone_alt,time_zone_generic,time_zone_num,time_zone_text,venue_id,venue_name,venue_short,website_url
0,Y,"401 East Jefferson Street&#xa;Phoenix, AZ&#x9;...",Phoenix,,N,401 East Jefferson Street,,,,AZ,...,109,MST,America/Phoenix,MST,-7,MST,15,Chase Field,Chase Field,dbacks.com
1,Y,"755 Battery Avenue&#xa;Atlanta, GA&#x9;30339",Atlanta,USA,N,755 Battery Avenue,,,,GA,...,144,ET,America/New_York,ET,-4,EDT,4705,SunTrust Park,SunTrust Park,braves.com
2,Y,"333 West Camden Street&#xa;Baltimore, MD&#x9;2...",Baltimore,,N,333 West Camden Street,,,,MD,...,110,ET,America/New_York,ET,-4,EDT,2,Oriole Park at Camden Yards,Oriole Park,orioles.com
3,Y,"4 Yawkey Way&#xa;Boston, MA&#x9;2215",Boston,,N,4 Yawkey Way,,,,MA,...,111,ET,America/New_York,ET,-4,EDT,3,Fenway Park,Fenway Park,redsox.com
4,Y,"1060 West Addison&#xa;Chicago, IL&#x9;60613-4397",Chicago,,N,1060 West Addison,,,,IL,...,112,CT,America/Chicago,CT,-5,CDT,17,Wrigley Field,Wrigley Field,cubs.com


In [137]:
teams_df = teams_df[
    [
        'mlb_org_id',
        'mlb_org',
        'city',
        'state',
        'division_full',
        'venue_name',
        'base_url'
    ]
]

teams_df.rename(columns = {
    'mlb_org_id': 'team_id',
    'mlb_org': 'team_name',
    'division_full': 'division'
}, inplace=True)

teams_df.sort_values(['division', 'team_name'], inplace=True)
teams_df.head()

,team_id,team_name,city,state,division,venue_name,base_url
5,145,Chicago White Sox,Chicago,IL,American League Central,Guaranteed Rate Field,whitesox.mlb.com
7,114,Cleveland Indians,Cleveland,OH,American League Central,Progressive Field,indians.mlb.com
9,116,Detroit Tigers,Detroit,MI,American League Central,Comerica Park,tigers.mlb.com
11,118,Kansas City Royals,Kansas City,MO,American League Central,Kauffman Stadium,royals.mlb.com
16,142,Minnesota Twins,Minneapolis,MN,American League Central,Target Field,twins.mlb.com


In [138]:
teams_df.set_index(['division', 'team_name'], inplace=True)
teams_df

team_id           city state  \
division                team_name                                            
American League Central Chicago White Sox         145        Chicago    IL   
                        Cleveland Indians         114      Cleveland    OH   
                        Detroit Tigers            116        Detroit    MI   
                        Kansas City Royals        118    Kansas City    MO   
                        Minnesota Twins           142    Minneapolis    MN   
American League East    Baltimore Orioles         110      Baltimore    MD   
                        Boston Red Sox            111         Boston    MA   
                        New York Yankees          147          Bronx    NY   
                        Tampa Bay Rays            139      Tampa Bay    FL   
                        Toronto Blue Jays         141        Toronto    ON   
American League West    Houston Astros            117        Houston    TX   
                        Los Angeles Angels        108        Anaheim    CA   
                        Oakland Athletics         133        Oakland    CA   
                        Seattle Mariners          136        Seattle    WA   
                        Texas Rangers             140      Arlington    TX   
National League Central Chicago Cubs              112        Chicago    IL   
                        Cincinnati Reds           113     Cincinnati    OH   
                        Milwaukee Brewers         158      Milwaukee    WI   
                        Pittsburgh Pirates        134     Pittsburgh    PA   
                        St. Louis Cardinals       138      St. Louis    MO   
National League East    Atlanta Braves            144        Atlanta    GA   
                        Miami Marlins             146          Miami    FL   
                        New York Mets             121       New York    NY   
                        Philadelphia Phillies     143   Philadelphia    PA   
                        Washington Nationals      120     Washington    DC   
National League West    Arizona Diamondbacks      109        Phoenix    AZ   
                        Colorado Rockies          115         Denver    CO   
                        Los Angeles Dodgers       119    Los Angeles    CA   
                        San Diego Padres          135      San Diego    CA   
                        San Francisco Giants      137  San Francisco    CA   

                                                                 venue_name  \
division                team_name                                             
American League Central Chicago White Sox             Guaranteed Rate Field   
                        Cleveland Indians                 Progressive Field   
                        Detroit Tigers                        Comerica Park   
                        Kansas City Royals                 Kauffman Stadium   
                        Minnesota Twins                        Target Field   
American League East    Baltimore Orioles       Oriole Park at Camden Yards   
                        Boston Red Sox                          Fenway Park   
                        New York Yankees                     Yankee Stadium   
                        Tampa Bay Rays                      Tropicana Field   
                        Toronto Blue Jays                     Rogers Centre   
American League West    Houston Astros                     Minute Maid Park   
                        Los Angeles Angels                    Angel Stadium   
                        Oakland Athletics                  Oakland Coliseum   
                        Seattle Mariners                      T-Mobile Park   
                        Texas Rangers          Globe Life Park in Arlington   
National League Central Chicago Cubs                          Wrigley Field   
                        Cincinnati Reds            Great American Ball Park   
                        Milwaukee Brewers       

In [139]:
teams_df_transposed = teams_df.T
teams_df_transposed

division   American League Central                                     \
team_name        Chicago White Sox  Cleveland Indians  Detroit Tigers   
team_id                        145                114             116   
city                       Chicago          Cleveland         Detroit   
state                           IL                 OH              MI   
venue_name   Guaranteed Rate Field  Progressive Field   Comerica Park   
base_url          whitesox.mlb.com    indians.mlb.com  tigers.mlb.com   

division                                              American League East  \
team_name  Kansas City Royals Minnesota Twins            Baltimore Orioles   
team_id                   118             142                          110   
city              Kansas City     Minneapolis                    Baltimore   
state                      MO              MN                           MD   
venue_name   Kauffman Stadium    Target Field  Oriole Park at Camden Yards   
base_url       royals.mlb.com   twins.mlb.com              orioles.mlb.com   

division                                                      \
team_name   Boston Red Sox New York Yankees   Tampa Bay Rays   
team_id                111              147              139   
city                Boston            Bronx        Tampa Bay   
state                   MA               NY               FL   
venue_name     Fenway Park   Yankee Stadium  Tropicana Field   
base_url    redsox.mlb.com  yankees.mlb.com     rays.mlb.com   

division                             ...          National League East  \
team_name  Toronto Blue Jays         ...                Atlanta Braves   
team_id                  141         ...                           144   
city                 Toronto         ...                       Atlanta   
state                     ON         ...                            GA   
venue_name     Rogers Centre         ...                 SunTrust Park   
base_url    bluejays.mlb.com         ...                braves.mlb.com   

division                                                         \
team_name     Miami Marlins New York Mets Philadelphia Phillies   
team_id                 146           121                   143   
city                  Miami      New York          Philadelphia   
state                    FL            NY                    PA   
venue_name     Marlins Park    Citi Field    Citizens Bank Park   
base_url    marlins.mlb.com  mets.mlb.com      phillies.mlb.com   

division                        National League West                   \
team_name  Washington Nationals Arizona Diamondbacks Colorado Rockies   
team_id                     120                  109              115   
city                 Washington              Phoenix           Denver   
state                        DC                   AZ               CO   
venue_name       Nationals Park          Chase Field      Coors Field   
base_url      nationals.mlb.com       dbacks.mlb.com  rockies.mlb.com   

division                                                              
team_name  Los Angeles Dodgers San Diego Padres San Francisco Giants  
team_id                    119              135                  137  
city               Los Angeles        San Diego        San Francisco  
state                       CA               CA                   CA  
venue_name      Dodger Stadium       Petco Park          Oracle Park  
base_url       dodgers.mlb.com   padres.mlb.com       giants.mlb.com  

[5 rows x 30 columns]

In [140]:
teams_df_transposed_2 = teams_df_transposed.copy()
teams_df_transposed_2.columns = teams_df_transposed_2.columns.droplevel()
teams_df_transposed_2

team_name,Chicago White Sox,Cleveland Indians,Detroit Tigers,Kansas City Royals,Minnesota Twins,Baltimore Orioles,Boston Red Sox,New York Yankees,Tampa Bay Rays,Toronto Blue Jays,...,Atlanta Braves,Miami Marlins,New York Mets,Philadelphia Phillies,Washington Nationals,Arizona Diamondbacks,Colorado Rockies,Los Angeles Dodgers,San Diego Padres,San Francisco Giants
team_id,145,114,116,118,142,110,111,147,139,141,...,144,146,121,143,120,109,115,119,135,137
city,Chicago,Cleveland,Detroit,Kansas City,Minneapolis,Baltimore,Boston,Bronx,Tampa Bay,Toronto,...,Atlanta,Miami,New York,Philadelphia,Washington,Phoenix,Denver,Los Angeles,San Diego,San Francisco
state,IL,OH,MI,MO,MN,MD,MA,NY,FL,ON,...,GA,FL,NY,PA,DC,AZ,CO,CA,CA,CA
venue_name,Guaranteed Rate Field,Progressive Field,Comerica Park,Kauffman Stadium,Target Field,Oriole Park at Camden Yards,Fenway Park,Yankee Stadium,Tropicana Field,Rogers Centre,...,SunTrust Park,Marlins Park,Citi Field,Citizens Bank Park,Nationals Park,Chase Field,Coors Field,Dodger Stadium,Petco Park,Oracle Park
base_url,whitesox.mlb.com,indians.mlb.com,tigers.mlb.com,royals.mlb.com,twins.mlb.com,orioles.mlb.com,redsox.mlb.com,yankees.mlb.com,rays.mlb.com,bluejays.mlb.com,...,braves.mlb.com,marlins.mlb.com,mets.mlb.com,phillies.mlb.com,nationals.mlb.com,dbacks.mlb.com,rockies.mlb.com,dodgers.mlb.com,padres.mlb.com,giants.mlb.com


In [141]:
teams_dict = {}
teams_dict_values = [pd.DataFrame(teams_df_transposed_2.iloc[:,0]).to_dict('dict')]
for x in range(1, len(teams_df_transposed.columns)):
    if teams_df_transposed.columns[x][0] == teams_df_transposed.columns[x-1][0]:
        teams_dict_values.append(pd.DataFrame(teams_df_transposed_2.iloc[:,x]).to_dict('dict'))
    else:
        teams_dict[teams_df_transposed.columns[x-1][0]] = teams_dict_values
        teams_dict_values = [pd.DataFrame(teams_df_transposed_2.iloc[:,x]).to_dict('dict')]
teams_dict[teams_df_transposed.columns[len(teams_df_transposed.columns)-1][0]] = teams_dict_values
pprint(teams_dict)

{'American League Central': [{'Chicago White Sox': {'base_url': 'whitesox.mlb.com',
                                                    'city': 'Chicago',
                                                    'state': 'IL',
                                                    'team_id': '145',
                                                    'venue_name': 'Guaranteed '
                                                                  'Rate '
                                                                  'Field'}},
                             {'Cleveland Indians': {'base_url': 'indians.mlb.com',
                                                    'city': 'Cleveland',
                                                    'state': 'OH',
                                                    'team_id': '114',
                                                    'venue_name': 'Progressive '
                                                                  'Field'}},
                          

In [142]:
print(teams_dict.keys())

dict_keys(['American League Central', 'American League East', 'American League West', 'National League Central', 'National League East', 'National League West'])


In [145]:
for team in teams_dict['National League West']:
    for key, value in team.items():
        if key == 'Colorado Rockies':
            rox_id = team[key]['team_id']

rox_id

'115'

In [148]:
rox_roster_endpoint = f"/json/named.roster_40.bam?team_id={rox_id}"
rox_roster_response = requests.get(base_url + rox_roster_endpoint)
print(rox_roster_response)
print(rox_roster_response.url)

<Response [200]>
http://lookup-service-prod.mlb.com/json/named.roster_40.bam?team_id=115


In [150]:
rox_roster_data = rox_roster_response.json()
pprint(rox_roster_data)

{'roster_40': {'copyRight': ' Copyright 2019 MLB Advanced Media, L.P.  Use of '
                            'any content on this page acknowledges agreement '
                            'to the terms posted here '
                            'http://gdx.mlb.com/components/copyright.txt  ',
               'queryResults': {'created': '2019-05-15T07:06:05',
                                'row': [{'bats': 'S',
                                         'birth_date': '1994-06-04T00:00:00',
                                         'college': '',
                                         'end_date': '',
                                         'height_feet': '6',
                                         'height_inches': '5',
                                         'jersey_number': '62',
                                         'name_display_first_last': 'Yency '
                                                                    'Almonte',
                                         'name_displa

                                         'team_name': 'Colorado Rockies',
                                         'throws': 'L',
                                         'weight': '185'},
                                        {'bats': 'R',
                                         'birth_date': '1995-04-30T00:00:00',
                                         'college': '',
                                         'end_date': '',
                                         'height_feet': '6',
                                         'height_inches': '4',
                                         'jersey_number': '32',
                                         'name_display_first_last': 'Jesus '
                                                                    'Tinoco',
                                         'name_display_last_first': 'Tinoco, '
                                                                    'Jesus',
                                         'name_first': 'Jesus',
   

In [159]:
rox_roster_data_for_df = rox_roster_data['roster_40']['queryResults']['row']

In [160]:
rox_roster_df = pd.DataFrame(rox_roster_data_for_df)
rox_roster_df.head()

,bats,birth_date,college,end_date,height_feet,height_inches,jersey_number,name_display_first_last,name_display_last_first,name_first,...,pro_debut_date,start_date,starter_sw,status_code,team_abbrev,team_code,team_id,team_name,throws,weight
0,S,1994-06-04T00:00:00,,,6,5,62,Yency Almonte,"Almonte, Yency",Yency,...,2018-06-21T00:00:00,2016-11-18T00:00:00,N,A,COL,col,115,Colorado Rockies,R,217
1,L,1989-12-30T00:00:00,Oregon,,6,3,44,Tyler Anderson,"Anderson, Tyler",Tyler,...,2016-06-12T00:00:00,2014-11-20T00:00:00,N,RM,COL,col,115,Colorado Rockies,L,215
2,R,1991-04-16T00:00:00,,,6,2,28,Nolan Arenado,"Arenado, Nolan",Nolan,...,2013-04-28T00:00:00,2013-04-28T00:00:00,N,A,COL,col,115,Colorado Rockies,R,215
3,R,1989-04-26T00:00:00,Texas Tech,,6,0,35,Chad Bettis,"Bettis, Chad",Chad,...,2013-08-01T00:00:00,2013-08-01T00:00:00,N,A,COL,col,115,Colorado Rockies,R,201
4,L,1986-07-01T00:00:00,Georgia Tech,,6,3,19,Charlie Blackmon,"Blackmon, Charlie",Charles,...,2011-06-07T00:00:00,2011-06-07T00:00:00,N,A,COL,col,115,Colorado Rockies,L,220


In [161]:
rox_roster_df = rox_roster_df[
    [
        'name_display_last_first',
        'player_id',
        'jersey_number',
        'birth_date',
        'college',
        'pro_debut_date',
        'position_txt',
        'bats',
        'throws'
    ]
]

rox_roster_df.rename(columns = {
    'name_display_last_first': 'name',
    'position_txt': 'position',
}, inplace=True)

rox_roster_df.sort_values(['position', 'name'], inplace=True)
rox_roster_df.head()

,name,player_id,jersey_number,birth_date,college,pro_debut_date,position,bats,throws
25,"Murphy, Daniel",502517,9,1985-04-01T00:00:00,Jacksonville,2008-08-02T00:00:00,1B,L,R
30,"Reynolds, Mark",448602,12,1983-08-03T00:00:00,Virginia,2007-05-16T00:00:00,1B,R,R
16,"Hampson, Garrett",641658,1,1994-10-10T00:00:00,Long Beach State,2018-07-21T00:00:00,2B,R,R
24,"McMahon, Ryan",641857,24,1994-12-14T00:00:00,,2017-08-12T00:00:00,2B,L,R
2,"Arenado, Nolan",571448,28,1991-04-16T00:00:00,,2013-04-28T00:00:00,3B,R,R


In [162]:
rox_roster_df.set_index(['position', 'name'], inplace=True)
rox_roster_df.head()

player_id jersey_number           birth_date  \
position name                                                            
1B       Murphy, Daniel      502517             9  1985-04-01T00:00:00   
         Reynolds, Mark      448602            12  1983-08-03T00:00:00   
2B       Hampson, Garrett    641658             1  1994-10-10T00:00:00   
         McMahon, Ryan       641857            24  1994-12-14T00:00:00   
3B       Arenado, Nolan      571448            28  1991-04-16T00:00:00   

                                    college       pro_debut_date bats throws  
position name                                                                 
1B       Murphy, Daniel        Jacksonville  2008-08-02T00:00:00    L      R  
         Reynolds, Mark            Virginia  2007-05-16T00:00:00    R      R  
2B       Hampson, Garrett  Long Beach State  2018-07-21T00:00:00    R      R  
         McMahon, Ryan                       2017-08-12T00:00:00    L      R  
3B       Arenado, Nolan                      2013-04-28T00:00:00    R      R

In [168]:
rox_roster_df['birth_date'] = rox_roster_df['birth_date'].apply(lambda x: x.split('T')[0])
rox_roster_df['pro_debut_date'] = rox_roster_df['pro_debut_date'].apply(lambda x: x.split('T')[0])

rox_roster_df

player_id jersey_number  birth_date  \
position name                                                     
1B       Murphy, Daniel        502517             9  1985-04-01   
         Reynolds, Mark        448602            12  1983-08-03   
2B       Hampson, Garrett      641658             1  1994-10-10   
         McMahon, Ryan         641857            24  1994-12-14   
3B       Arenado, Nolan        571448            28  1991-04-16   
         Fuentes, Josh         658069             8  1993-02-19   
         Valaika, Pat          642162             4  1992-09-09   
C        Iannetta, Chris       455104            22  1983-04-08   
         Wolters, Tony         547172            14  1992-06-09   
CF       Daza, Yonathan        602074            31  1994-02-28   
         Desmond, Ian          435622            20  1985-09-20   
         Tapia, Raimel         606132            15  1994-02-04   
LF       Dahl, David           621311            26  1994-04-01   
P        Almonte, Yency        622075            62  1994-06-04   
         Anderson, Tyler       542881            44  1989-12-30   
         Bettis, Chad          518452            35  1989-04-26   
         Castellani, Ryan      624418            60  1996-04-01   
         Davis, Wade           451584            71  1985-09-07   
         Dunn, Mike            445197            38  1985-05-23   
         Estevez, Carlos       608032            54  1992-12-28   
         Freeland, Kyle        607536            21  1993-05-14   
         Gray, Jon             592351            55  1991-11-05   
         Hoffman, Jeff         656546            34  1993-01-08   
         Johnson, DJ           597113            63  1989-08-30   
         Lawrence, Justin      664875            74  1994-11-25   
         Marquez, German       608566            48  1995-02-22   
         McGee, Jake           459429            51  1986-08-06   
         Musgrave, Harrison    643466            59  1992-03-03   
         Oberg, Scott          623184            45  1990-03-13   
         Oh, Seunghwan         493200            18  1982-07-15   
         Pazos, James          572021                1991-05-05   
         Rusin, Chris          543734            52  1986-10-22   
         Senzatela, Antonio    622608            49  1995-01-21   
         Shaw, Bryan           543766            29  1987-11-08   
         Tinoco, Jesus         622786            32  1995-04-30   
RF       Blackmon, Charlie     453568            19  1986-07-01   
         Cuevas, Noel          571595            56  1991-10-02   
         Hilliard, Sam         656541            43  1994-02-21   
SS       Story, Trevor         596115            27  1992-11-15   

                                                             college  \
position name                                                          
1B       Murphy, Daniel                                 Jacksonville   
         Reynolds, Mark                                     Virginia   
2B       Hampson, Garrett                           Long Beach State   
         McMahon, Ryan                                                 
3B       Arenado, Nolan                                                
         Fuentes, Josh                              Missouri Baptist   
         Valaika, Pat                                           UCLA   
C        Iannetta, Chris                              North Carolina   
         Wolters, Tony                                                 
CF       Daza, Yonathan                                                
         Desmond, Ian                                                  
         Tapia, Raimel                                                 
LF       Dahl, David                                                   
P        Almonte, Yency                                                
         Anderson, Tyler                                      Oregon   
         Bettis, Chad                                   

In [170]:
rox_roster_df_transposed = rox_roster_df.T
rox_roster_df_transposed

position                   1B                               2B                \
name           Murphy, Daniel Reynolds, Mark  Hampson, Garrett McMahon, Ryan   
player_id              502517         448602            641658        641857   
jersey_number               9             12                 1            24   
birth_date         1985-04-01     1983-08-03        1994-10-10    1994-12-14   
college          Jacksonville       Virginia  Long Beach State                 
pro_debut_date     2008-08-02     2007-05-16        2018-07-21    2017-08-12   
bats                        L              R                 R             L   
throws                      R              R                 R             R   

position                   3B                                              C  \
name           Arenado, Nolan     Fuentes, Josh Valaika, Pat Iannetta, Chris   
player_id              571448            658069       642162          455104   
jersey_number              28                 8            4              22   
birth_date         1991-04-16        1993-02-19   1992-09-09      1983-04-08   
college                        Missouri Baptist         UCLA  North Carolina   
pro_debut_date     2013-04-28        2019-04-06   2016-09-06      2006-08-27   
bats                        R                 R            R               R   
throws                      R                 R            R               R   

position                                 CF      ...                        P  \
name           Wolters, Tony Daza, Yonathan      ...            Oh, Seunghwan   
player_id             547172         602074      ...                   493200   
jersey_number             14             31      ...                       18   
birth_date        1992-06-09     1994-02-28      ...               1982-07-15   
college                                          ...       Dankook University   
pro_debut_date    2016-04-05     2019-04-09      ...               2016-04-03   
bats                       L              R      ...                        R   
throws                     R              R      ...                        R   

position                                                                       \
name           Pazos, James Rusin, Chris Senzatela, Antonio       Shaw, Bryan   
player_id            572021       543734             622608            543766   
jersey_number                         52                 49                29   
birth_date       1991-05-05   1986-10-22         1995-01-21        1987-11-08   
college           San Diego     Kentucky                     Long Beach State   
pro_debut_date   2015-09-05   2012-08-21         2017-04-06        2011-06-10   
bats                      R            L                  R                 S   
throws                    L            L                  R                 R   

position                                    RF  \
name           Tinoco, Jesus Blackmon, Charlie   
player_id             622786            453568   
jersey_number             32                19   
birth_date        1995-04-30        1986-07-01   
college                           Georgia Tech   
pro_debut_date                      2011-06-07   
bats                       R                 L   
throws                     R                 L   

position                                                                \
name                                       Cuevas, Noel  Hilliard, Sam   
player_id                                        571595         656541   
jersey_number                                        56             43   
birth_date                                   1991-10-02     1994-02-21   
college         Interamerican University of Puerto Rico  Wichita State   
pro_debut_date                               2018-04-22                  
bats                                                  R              L   
throws                                                R

In [171]:
rox_roster_df_transposed_2 = rox_roster_df_transposed.copy()
rox_roster_df_transposed_2.columns = rox_roster_df_transposed_2.columns.droplevel()
rox_roster_df_transposed_2

name,"Murphy, Daniel","Reynolds, Mark","Hampson, Garrett","McMahon, Ryan","Arenado, Nolan","Fuentes, Josh","Valaika, Pat","Iannetta, Chris","Wolters, Tony","Daza, Yonathan",...,"Oh, Seunghwan","Pazos, James","Rusin, Chris","Senzatela, Antonio","Shaw, Bryan","Tinoco, Jesus","Blackmon, Charlie","Cuevas, Noel","Hilliard, Sam","Story, Trevor"
player_id,502517,448602,641658,641857,571448,658069,642162,455104,547172,602074,...,493200,572021,543734,622608,543766,622786,453568,571595,656541,596115
jersey_number,9,12,1,24,28,8,4,22,14,31,...,18,,52,49,29,32,19,56,43,27
birth_date,1985-04-01,1983-08-03,1994-10-10,1994-12-14,1991-04-16,1993-02-19,1992-09-09,1983-04-08,1992-06-09,1994-02-28,...,1982-07-15,1991-05-05,1986-10-22,1995-01-21,1987-11-08,1995-04-30,1986-07-01,1991-10-02,1994-02-21,1992-11-15
college,Jacksonville,Virginia,Long Beach State,,,Missouri Baptist,UCLA,North Carolina,,,...,Dankook University,San Diego,Kentucky,,Long Beach State,,Georgia Tech,Interamerican University of Puerto Rico,Wichita State,
pro_debut_date,2008-08-02,2007-05-16,2018-07-21,2017-08-12,2013-04-28,2019-04-06,2016-09-06,2006-08-27,2016-04-05,2019-04-09,...,2016-04-03,2015-09-05,2012-08-21,2017-04-06,2011-06-10,,2011-06-07,2018-04-22,,2016-04-04
bats,L,R,R,L,R,R,R,R,L,R,...,R,R,L,R,S,R,L,R,L,R
throws,R,R,R,R,R,R,R,R,R,R,...,R,L,L,R,R,R,L,R,L,R


In [172]:
rox_roster_dict = {}
rox_roster_dict_values = [pd.DataFrame(rox_roster_df_transposed_2.iloc[:,0]).to_dict('dict')]
for x in range(1, len(rox_roster_df_transposed.columns)):
    if rox_roster_df_transposed.columns[x][0] == rox_roster_df_transposed.columns[x-1][0]:
        rox_roster_dict_values.append(pd.DataFrame(rox_roster_df_transposed_2.iloc[:,x]).to_dict('dict'))
    else:
        rox_roster_dict[rox_roster_df_transposed.columns[x-1][0]] = rox_roster_dict_values
        rox_roster_dict_values = [pd.DataFrame(rox_roster_df_transposed_2.iloc[:,x]).to_dict('dict')]
rox_roster_dict[rox_roster_df_transposed.columns[len(rox_roster_df_transposed.columns)-1][0]] = rox_roster_dict_values
pprint(rox_roster_dict)

{'1B': [{'Murphy, Daniel': {'bats': 'L',
                            'birth_date': '1985-04-01',
                            'college': 'Jacksonville',
                            'jersey_number': '9',
                            'player_id': '502517',
                            'pro_debut_date': '2008-08-02',
                            'throws': 'R'}},
        {'Reynolds, Mark': {'bats': 'R',
                            'birth_date': '1983-08-03',
                            'college': 'Virginia',
                            'jersey_number': '12',
                            'player_id': '448602',
                            'pro_debut_date': '2007-05-16',
                            'throws': 'R'}}],
 '2B': [{'Hampson, Garrett': {'bats': 'R',
                              'birth_date': '1994-10-10',
                              'college': 'Long Beach State',
                              'jersey_number': '1',
                              'player_id': '641658',
                   